<a href="https://colab.research.google.com/github/habilg/keras_advanced/blob/master/keras_for_researchers_08_CustumTrainingLoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps:
> prepare Data by data api. Dont forget shuffle, batch and...  
> select a loss  
> select an optimizer  
> put every thing in Gradient Tape
> inside gradianttape,
+ feed the batch to model
+ calculate the loss   
> claculate the calculus to the model trainable weights  
> update the model weights by optimizers

>
```python
for epoch in range(epochs):
  
  for batch,data in eumerate(dataset)
    with GradientTape as g:
      y_pred=model(data.x_train, Training=True)
      loss_value=loss_fn(data.y_train, y_pred)
    metric.update_state(data.y_train, y_pred)
    grads=g.gradient(loss_value,model.trainable_weights)
    opt.apply_gradients(zip(grads,model.trainable_weights))
  #return the metric value for epoch(mean of all metrics for batchs)
  train_mae_val=metric.result()
  # reset the metric for the next epoch
  train_mae_val.reset_states()


```


In [16]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense

In [17]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [18]:
housing = fetch_california_housing()
x_train, x_test, y_train, y_test = train_test_split(housing["data"], housing["target"])
training_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
training_data=training_data.batch(30)

In [19]:
model=keras.models.Sequential([
    Dense(50,activation="relu"),
    Dense(10,activation="relu"),
    Dense(1)])

In [26]:
loss_fn=keras.losses.MeanSquaredError()
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
metric=tf.keras.metrics.MeanSquaredError()

In [35]:
epochs=2
for epoch in range(epochs):
  epoch_loss=[]
  for step,(xt,yt) in enumerate(training_data):
    with tf.GradientTape() as tape:
      model_out=model(xt, training=True)
      loss_value=loss_fn(yt, model_out)

    grads=tape.gradient(loss_value,model.trainable_variables)
    opt.apply_gradients(zip(grads,model.trainable_variables))
    metric.update_state(yt, model_out)
    if step% 100 ==0:
      print(f"epoch {epoch+1} form {epochs} epochs: batch {step} Training loss: ",float(loss_value),"Metric: ",float(metric.result()))
    epoch_loss.append(loss_value)
  print("**********epoch_loss: ",tf.math.reduce_mean(tf.convert_to_tensor(epoch_loss, dtype=tf.float64).numpy()))
  print("**********metric: ",float(metric.result()))
  metric.reset_state()

epoch 1 form 2 epochs: batch 0 Training loss:  0.6834205985069275 Metric:  0.6834205985069275
epoch 1 form 2 epochs: batch 100 Training loss:  0.5277000665664673 Metric:  0.8134132027626038
epoch 1 form 2 epochs: batch 200 Training loss:  10.470314979553223 Metric:  1.9835187196731567
epoch 1 form 2 epochs: batch 300 Training loss:  0.8241281509399414 Metric:  3.6277568340301514
epoch 1 form 2 epochs: batch 400 Training loss:  0.29409071803092957 Metric:  2.958096742630005
epoch 1 form 2 epochs: batch 500 Training loss:  0.9429605603218079 Metric:  2.511659860610962
**********epoch_loss:  tf.Tensor(2.460677313307921, shape=(), dtype=float64)
**********metric:  2.4606754779815674
epoch 2 form 2 epochs: batch 0 Training loss:  0.6835720539093018 Metric:  0.6835720539093018
epoch 2 form 2 epochs: batch 100 Training loss:  0.514913022518158 Metric:  0.8267788290977478
epoch 2 form 2 epochs: batch 200 Training loss:  32.73588562011719 Metric:  4.246690273284912
epoch 2 form 2 epochs: batch 